In [182]:
import numpy as np
import matplotlib.pyplot as plt
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import pandas as pd

In [183]:
import pickle
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic
import os

In [184]:
def get_all_filenames(folder_path):
    filenames = []
    for filename in os.listdir(folder_path):
        filenames.append(filename)
    return filenames

In [185]:
folder_path = "D:\Spatial_interpolation\SSIN\data\Air_quality-Shenzhen"
all_filenames = get_all_filenames(folder_path)

In [186]:
file_result=[]
for filename in all_filenames:
    x=pd.read_csv(folder_path+"/"+filename)
    x=np.array(x)
    x=np.reshape(x,(7296, 6 ,1))
    file_result.append(x)

In [187]:
result=file_result[0]

In [188]:
for i in range(1,len(file_result)):
    result = np.append(result,file_result[i] , axis=2)

In [189]:
X_raw=result

In [190]:
j=5
train_data=X_raw[:,j,:]
columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(train_data, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [191]:
missing_columns

array([], dtype=int64)

In [192]:
result.shape

(7296, 10)

In [193]:
# train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
# test_list=[43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]
train_list=[0,1,2,3,4,5]
test_list=[6,7,8,9]

In [194]:
train_data=result[:,train_list]
test_data=result[:,test_list]

In [195]:
len(train_data[0])

6

In [196]:
train_data

array([[0.00222222, 0.01777778, 0.03333333, 0.01333333, 0.00888889,
        0.        ],
       [0.00222222, 0.01111111, 0.02      , 0.00444444, 0.00222222,
        0.        ],
       [0.        , 0.00888889, 0.03111111, 0.00444444, 0.00222222,
        0.        ],
       ...,
       [0.02666667, 0.00444444, 0.        , 0.01333333, 0.00666667,
        0.01555556],
       [0.02666667, 0.00222222, 0.        , 0.01333333, 0.01111111,
        0.00666667],
       [0.02666667, 0.00222222, 0.        , 0.01333333, 0.00666667,
        0.00666667]])

In [197]:
train_data = np.reshape(train_data, (7296, 6,1))
test_data = np.reshape(test_data, (7296, 4,1))

In [198]:
train_data

array([[[0.00222222],
        [0.01777778],
        [0.03333333],
        [0.01333333],
        [0.00888889],
        [0.        ]],

       [[0.00222222],
        [0.01111111],
        [0.02      ],
        [0.00444444],
        [0.00222222],
        [0.        ]],

       [[0.        ],
        [0.00888889],
        [0.03111111],
        [0.00444444],
        [0.00222222],
        [0.        ]],

       ...,

       [[0.02666667],
        [0.00444444],
        [0.        ],
        [0.01333333],
        [0.00666667],
        [0.01555556]],

       [[0.02666667],
        [0.00222222],
        [0.        ],
        [0.01333333],
        [0.01111111],
        [0.00666667]],

       [[0.02666667],
        [0.00222222],
        [0.        ],
        [0.01333333],
        [0.00666667],
        [0.00666667]]])

In [199]:
stations = pd.read_csv("D:\Spatial_interpolation\SSIN\data\location_of_Shenzhen.csv")

In [200]:
df_train=stations.iloc[0:6]
df_test=stations.iloc[6:10]

In [201]:
# train_info=np.c_[df_train["lat"],df_train["lon"]]
# test_info=np.c_[df_test["lat"],df_test["lon"]]

In [202]:
# xls = np.linspace(0,100,100)
# yls = np.linspace(0,100,100)
# xgrid,ygrid = np.meshgrid(xls,yls)
# OK = OrdinaryKriging(
#     data[:, 0],
#     data[:, 1],
#     data[:, 2],
#     variogram_model="linear",
#     verbose=False,
#     enable_plotting=False,
# )

In [203]:
result=[]
for i in range(len(train_data)):
    OK = OrdinaryKriging(
    df_train["lat"],
    df_train["lon"],
    train_data[i],
    variogram_model="linear",
    verbose=False,
    enable_plotting=False,
    )
    z, ss = OK.execute("points", df_test["lat"], df_test["lon"])
    result.append(z)

In [204]:

test_data = np.reshape(test_data, (29184))

In [205]:
result=np.array(result)

In [206]:
result=np.reshape(result, (29184))

## MAE

In [207]:
sum(abs(test_data-result))/(7296*4)

0.010816361267112893

## RMSE

In [208]:
import math
math.sqrt(sum((test_data-result)**2)/(7296*4))

0.01893948585322329

### 另外的参数

In [209]:
result=[]
for i in range(len(train_data)):
    OK = OrdinaryKriging(
    df_train["lat"],
    df_train["lon"],
    train_data[i],
    variogram_model="gaussian",
    verbose=False,
    enable_plotting=False,
    )
    z, ss = OK.execute("points", df_test["lat"], df_test["lon"])
    result.append(z)

In [210]:
result=np.array(result)
result=np.reshape(result, (29184))

In [211]:
sum(abs(test_data-result))/(7296*4)

0.010348189948649292

In [212]:
math.sqrt(sum((test_data-result)**2)/(7296*4))

0.018375626492026495

## SSIN

In [213]:
# result_SSIN=pd.read_csv('test_ret.csv')

## MAE

In [214]:
# sum(abs(result_SSIN['pred']-result_SSIN["label"]))/len(result_SSIN)

## RMSE

In [215]:
# math.sqrt(sum((result_SSIN['pred']-result_SSIN["label"])**2)/len(result_SSIN))

## kriging